In [26]:
import torch
from utils import LQR_2D_true_solution, out_put, LQR_1D_true_solution, l_2_compute_1D_Q, l_2_compute_1D_V, true_V_eval_2D
from main_phibe import phibe_finder_1D_LQR, phibe_finder_2D_LQR
from bases import bases_poly, d_bases_poly, sec_bases_poly, bases_2d, bases_poly_simp, d_bases_poly_simp, sec_bases_poly_simp, bases_2d_simp, bases_poly_2D, bases_poly_2D_s_a, d_bases_poly_2D, bases_poly_2D_simp, d_bases_poly_2D_simp, sec_d_bases_poly_2D, sec_d_bases_poly_2D_simp
import matplotlib.pyplot as plt
from math import sqrt
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)
device = torch.device("cpu")

In [3]:
# 1D case

In [3]:
def reward(s, a):
    return - Q * (s**2) - R * (a**2)

In [36]:
A = 1
B = 0.34
sig = 0.
R = 1
Q = 1
S = 0
beta = 0
M = 2
info_true = {"A":A, "B":B, "sig":sig, "R":R, "Q":Q,}
true_b, True_V = LQR_1D_true_solution(A, B, Q, R, sig, beta)
print(true_b, True_V)

-6.047704923884915 tensor([  0.0000,   0.0000, -17.7874])


In [37]:
b_init = - 8
Q_init = torch.tensor([1., -1., -1., -1.]).to(device)
batch_size = int(1e6)
bd_low_s = -10
bd_upper_s = 10.
bd_low_b = -20.
bd_upper_b = 3.
bd_low_a = -30.
bd_upper_a = 30
bases_V = bases_poly_simp
d_bases_V = d_bases_poly_simp
sec_d_bases_V = sec_bases_poly_simp
bases_Q = bases_2d_simp
num_iter = 80
GD_num_iter = 80
I = 6
m = int(batch_size / I)
m_Q = int(batch_size / I)
lr = 0.01
dt = 1
order = 1
true_V = True_V
simp_basis = True
deterministic = True
b_val_samp_1, V_exact_dist_samp_1, V_func_samp_1 = phibe_finder_1D_LQR(beta, b_init, Q_init, bd_low_s, bd_upper_s, bd_low_b, bd_upper_b, reward, simp_basis,
                                                                                     bases_V, d_bases_V, sec_d_bases_V, bases_Q, num_iter, deterministic, "Galerkin", GD_num_iter, m, m_Q, I, lr, dt, order, 
                                                                                     true_V, info_true)
V_exact_dist_samp_1

Running Optimal Phibe of order 1 using Galerkin: 100%|██████████| 80/80 [00:03<00:00, 23.98it/s]


[10.9235954475780,
 0.590695047838741,
 0.00192410312269306,
 2.05458235174980e-8,
 0,
 0,
 0,
 7.00524512005031e-14,
 0,
 0,
 7.00524512005031e-14,
 7.00524512005031e-14,
 0,
 7.00524512005031e-14,
 0,
 0,
 0,
 0,
 0,
 0,
 3.50262256002516e-14,
 3.50262256002516e-14,
 7.00524512005031e-14,
 0,
 7.00524512005031e-14,
 0,
 0,
 0,
 3.50262256002516e-14,
 7.00524512005031e-14,
 0,
 0,
 7.00524512005031e-14,
 0,
 0,
 7.00524512005031e-14,
 7.00524512005031e-14,
 0,
 0,
 7.00524512005031e-14,
 7.00524512005031e-14,
 0,
 3.50262256002516e-14,
 7.00524512005031e-14,
 0,
 3.50262256002516e-14,
 3.50262256002516e-14,
 0,
 0,
 0,
 3.50262256002516e-14,
 0,
 0,
 7.00524512005031e-14,
 0,
 7.00524512005031e-14,
 0,
 7.00524512005031e-14,
 3.50262256002516e-14,
 0,
 0,
 7.00524512005031e-14,
 0,
 3.50262256002516e-14,
 7.00524512005031e-14,
 0,
 3.50262256002516e-14,
 0,
 0,
 0,
 0,
 3.50262256002516e-14,
 0,
 0,
 0,
 0,
 7.00524512005031e-14,
 0,
 0,
 7.00524512005031e-14]

In [38]:
print(b_val_samp_1)

[-8, tensor(-6.4244), tensor(-6.0681), tensor(-6.0478), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6.0477), tensor(-6

In [ ]:
# 2D case

In [4]:
def reward(traj_mat, act_mat):
    # traj_mat shape: (m, I, dim)
    # act_mat shape: (m, I, dim)
    # output shape: (m, I, 1)
    r_1 = torch.einsum("ijk,kl,ijl->ij", traj_mat, Q, traj_mat)
    r_2 = torch.einsum("ijk,kl,ijl->ij", act_mat, R, act_mat)
    return (- r_1 - r_2).unsqueeze(-1)

In [33]:
beta = 1e-15
A =  torch.inverse(0.2 * torch.tensor([[-3.0, 1.0], [1., -3.0]]))
B =  0.2 * torch.tensor([[3.0, 1.0], [1., 3.0]])
# C = - 0.1 * torch.tensor([[2.0, 1.0], [1., 1.0]])
# D = 0.1 * torch.tensor([[3.0, 2.0], [.5, 1.0]])
sig = 0.1
# Q = 1 * torch.tensor([[0.54, -0.86], [-0.86, 1.5]])
# R = 1 * torch.inverse(torch.tensor([[1.92, 0.4], [0.4, 0.3]]))
# Q = 1 * torch.tensor([[0.54, -0.26], [-0.26, 0.4]])
# R = 1 * torch.inverse(torch.tensor([[1.92, 0.4], [0.4, 0.3]]))
Q = 1 * torch.tensor([[0.9, 0.89], [0.89, 0.9]])
R = 1 * torch.inverse(torch.tensor([[.4, 0.398], [0.398, 0.4]]))

In [34]:
A_norm = torch.sqrt(torch.abs(torch.linalg.eig(A.T @ A).eigenvalues).max())
A_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(A).T @ torch.inverse(A)).eigenvalues).max())
B_norm = torch.sqrt(torch.abs(torch.linalg.eig(B.T @ B).eigenvalues).max())
B_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(B).T @ torch.inverse(B)).eigenvalues).max())
print(A_norm)
print(A_inv_norm)
print(B_norm)
print(B_inv_norm)
print(A_norm**2 * A_inv_norm * B_norm * B_inv_norm**2)

tensor(2.5000)
tensor(0.8000)
tensor(0.8000)
tensor(2.5000)
tensor(25.0000)


In [35]:
Q_norm = torch.sqrt(torch.abs(torch.linalg.eig(Q.T @ Q).eigenvalues).max())
Q_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(Q).T @ torch.inverse(Q)).eigenvalues).max())
R_norm = torch.sqrt(torch.abs(torch.linalg.eig(R.T @ R).eigenvalues).max())
R_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(R).T @ torch.inverse(R)).eigenvalues).max())
print(Q_norm)
print(Q_inv_norm)
print(R_norm)
print(R_inv_norm)
print(Q_norm**2 * Q_inv_norm * R_norm * R_inv_norm**2)

tensor(1.7900)
tensor(100.0000)
tensor(500.0000)
tensor(0.7980)
tensor(102019.1848)


In [46]:
true_b, true_c, True_V = LQR_2D_true_solution(A, B, sig, Q, R, beta)
b_init = true_b + torch.randn(2, 2)
c_init = 2. * torch.rand(2, 1) - 1.
Q_init = 4. * torch.rand(10) - 2.
V_init = true_V_eval_2D(A, B, b_init, R, Q, beta, sig)

In [47]:
print(true_b)
print(true_c)
print(True_V)
print(V_init)

tensor([[-0.1104, -0.1104],
        [-0.1104, -0.1104]])
tensor([[0.],
        [0.]])
tensor([-3.4979e+12, -1.7489e-01, -3.4179e-01, -1.7489e-01])
tensor([-2.1272e+15, -1.8300e+01,  7.9402e+01, -1.9442e+02])


In [48]:
bd_low_s = -2
bd_upper_s = 2
bd_low_b = -2
bd_upper_b = 2
reward = reward
bases_V = bases_poly_2D
d_bases_V = d_bases_poly_2D
sec_d_bases_V = sec_d_bases_poly_2D
bases_Q = bases_poly_2D_s_a
num_iter = 20
GD_num_iter = 75
m=int(1e6)
m_Q=int(1e6)
I=6
I_Q=6
lr=0.1
order = 1
true_V = True_V
info_true = {"A":A, "B":B, "sig":sig, "R":R, "Q":Q}
dt = 10
simp_basis = False
deterministic = True
b_val_GD, V_exact_dist_1, V = phibe_finder_2D_LQR(beta, b_init, Q_init, bd_low_s, bd_upper_s, bd_low_b, bd_upper_b, reward, simp_basis, bases_V, d_bases_V, sec_d_bases_V, bases_Q, num_iter, deterministic, "Galerkin", GD_num_iter, m, m_Q, I, lr, dt, order, true_V, info_true)

Running Optimal Phibe of order 1 using Galerkin:   5%|▌         | 1/20 [00:01<00:34,  1.81s/it]

infeasibility detected


Running Optimal Phibe of order 1 using Galerkin:  10%|█         | 2/20 [00:03<00:28,  1.57s/it]

infeasibility detected


Running Optimal Phibe of order 1 using Galerkin:  15%|█▌        | 3/20 [00:04<00:25,  1.49s/it]

...

Running Optimal Phibe of order 1 using Galerkin:  85%|████████▌ | 17/20 [00:25<00:04,  1.45s/it]

infeasibility detected


Running Optimal Phibe of order 1 using Galerkin:  90%|█████████ | 18/20 [00:27<00:02,  1.45s/it]

infeasibility detected


Running Optimal Phibe of order 1 using Galerkin: 100%|██████████| 20/20 [00:30<00:00,  1.52s/it]


In [50]:
b_val_GD

[tensor([[-1.4470, -0.5010],
         [ 0.5295, -0.4738]]),
 tensor([[-40.9521,   1.5137],
         [-40.7309,   1.5027]]),
 tensor([[-21.5799,   0.1307],
         [-21.2122,  -0.1314]]),
 tensor([[-9.5046, -0.1030],
         [-9.6559,  0.0967]]),
 tensor([[-3.5995, -0.0838],
         [-3.5750, -0.0908]]),
 tensor([[-1.0077, -0.0938],
         [-1.0031, -0.0939]]),
 tensor([[-0.2053, -0.1072],
         [-0.2048, -0.1072]]),
 tensor([[-0.1117, -0.1103],
         [-0.1116, -0.1103]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
         [-0.1104, -0.1104]]),
 tensor([[-0.1104, -0.1104],
   